In [1]:
import numpy as np
import glob
import os
import cPickle as pickle
from collections import defaultdict
import sys

import shapefile as shp
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

import matplotlib.path as mplPath

import matplotlib.lines as mlines

import shutil
import seaborn as sns

from shutil import copyfile

%matplotlib inline

In [2]:
# eppy paramters
from eppy import modeleditor
from eppy.modeleditor import IDF
# set the idd file
iddfile = 'Energy+.idd'
IDF.setiddname(iddfile)

In [3]:
print len(glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_control.idf'))
print len(glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_experiment1.idf'))
print len(glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_experiment2.idf'))

1266
1266
1266


In [4]:
# find all the buildings with apartments
listFiles = glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_control.idf')

In [6]:
listApartments = []
for j,f in enumerate(listFiles):
    if j%500==0:
        print j
        sys.stdout.flush()
    idf1 = IDF(f)
    # find the uses
    people = idf1.idfobjects['PEOPLE']
    for pJ in range(len(people)):
        if 'AppM' in people[pJ].Number_of_People_Schedule_Name:
            listApartments.append(f)
            break
# save the apartment list
with open('../Intermediate_Results/listApartments.pickle', 'wb') as outFile:
    pickle.dump(listApartments, outFile, pickle.HIGHEST_PROTOCOL)

In [8]:
with open('../Intermediate_Results/listApartments.pickle', 'rb') as inFile:
    listApartments = pickle.load(inFile)

In [9]:
print 'There are', len(listApartments), 'buildings with apartments'

There are 1046 buildings with apartments


In [9]:
periodApartments = ['AppM_2','AppM_3','AppM_4']
# now in all the apartment idf files find the value
exampleApartments = []
for era in periodApartments:
    print era
    for f in listApartments:
        idf1 = IDF(f)
        constructions = idf1.idfobjects['CONSTRUCTION']
        if era in constructions[0].Name:
            # then get the insulation material name
            exampleApartments.append(f)
            break
with open('../Intermediate_Results/exampleEraApartments.pickle', 'wb') as outFile:
    pickle.dump(exampleApartments, outFile, pickle.HIGHEST_PROTOCOL)

In [10]:
with open('../Intermediate_Results/exampleEraApartments.pickle', 'rb') as inFile:
    exampleApartments = pickle.load(inFile)

In [11]:
exampleApartments

['../Idf_generated_files/All_buildings_standard_efficiency/0503082000_control.idf',
 '../Idf_generated_files/All_buildings_standard_efficiency/0501355000_control.idf',
 '../Idf_generated_files/All_buildings_standard_efficiency/0503131000_control.idf']

In [12]:
periodApartments = ['AppM_2','AppM_3','AppM_4']
searchMats = {}
# now for all the example apartment buildings, select the searchMats
for j,f in enumerate(exampleApartments):
    idf1 = IDF(f)
    constructions = idf1.idfobjects['CONSTRUCTION']
    searchMats[periodApartments[j]] = {}
    for con in constructions:
        #print con.Name
        if 'ExtWall' in con.Name:
            for obj in con['obj']:
                if 'Insul' in obj:
                    # then save the name
                    searchMats[periodApartments[j]]['Wall'] = obj
            break
    for con in constructions:
        if 'Roof' in con.Name:
            for obj in con['obj']:
                if 'Insul' in obj:
                    # then save the name
                    searchMats[periodApartments[j]]['Roof'] = obj
            break

In [13]:
# # delete the unwanted old idfs
# fileList = glob.glob('../Idf_generated_files/Apartment_insulation_increase/*.idf')
# print len(fileList)
# for file1 in fileList:
#     os.remove(file1)

0


In [16]:
# copy all the apartment buildings into a new folder
for j in range(len(listApartments)):
    idx = listApartments[j].split('iency/')[1].split('_')[0]
    fileList = glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*'+idx+'*')
    for f in fileList:
        if 'control' in f:
            newName = '../Idf_generated_files/Apartment_insulation_increase/'+idx+'_control_ins.idf'
            copyfile(f, newName)
        elif 'experiment1' in f:
            newName = '../Idf_generated_files/Apartment_insulation_increase/'+idx+'_experiment1_ins.idf'
            copyfile(f, newName)
        elif 'experiment2' in f:
            newName = '../Idf_generated_files/Apartment_insulation_increase/'+idx+'_experiment2_ins.idf'
            copyfile(f, newName)            
        else:
            print 'we have a problem', f

In [17]:
for j,f in enumerate(exampleApartments):
    idf1 = IDF(f)
    constructions = idf1.idfobjects['CONSTRUCTION']
    searchMats[periodApartments[j]] = {}
    for con in constructions:
        if 'ExtWall' in con.Name:
            for obj in con['obj']:
                if 'Insul' in obj:
                    # then save the name
                    searchMats[periodApartments[j]]['Wall'] = obj
            break
    for con in constructions:
        if 'Roof' in con.Name:
            for obj in con['obj']:
                if 'Insul' in obj:
                    # then save the name
                    searchMats[periodApartments[j]]['Roof'] = obj
            break

In [18]:
listFiles = glob.glob('../Idf_generated_files/Apartment_insulation_increase/*')
print len(listFiles)

457


In [19]:
targetValues = {}
targetValues['Roof'] = 0.1532*1.1
targetValues['Wall'] = 0.1192*1.1

In [21]:
problemFiles = []
for counter, f in enumerate(listFiles):
    if counter%100==0:
        print counter
        sys.stdout.flush()
    idf1 = IDF(f)
    # Find which era the apartment is from
    era = idf1.idfobjects['CONSTRUCTION'][0].Name.split('AppM_')[1][0]
    era = 'AppM_'+era
    # now change the insulation    
    materials = idf1.idfobjects['MATERIAL']
    upgrades = ['Wall', 'Roof']
    for upgrade in upgrades:
        found = 0
        changeMaterial = searchMats[era][upgrade]
        for j in range(len(materials)):
            if materials[j].Name == changeMaterial:
                materials[j].Thickness = targetValues[upgrade]
                #print materials[j]
                found+=1
                break
        if found == 0:
            print upgrade, f
            problemFiles.append([upgrade, f])
    idf1.save()
            

In [24]:
listFiles = glob.glob('../Idf_generated_files/Apartment_insulation_increase/*')